In [1]:
from bs4 import BeautifulSoup  # library utama untuk melakukan scrapping
import requests                # library untuk mendapatkan HTTP request
import unicodedata             # library untuk membersihkan unicode yang terbawa
import pandas as pd            # library untuk membuat data frame
import asyncio
import polars as pl
from tqdm import tqdm
import json

In [44]:
#Fungsi untuk mengambil data title dan link pada website dengan memasukkan baris data yang telah discraping dan barchart untuk loading
#saat get data
def get(row, bar, page, level):
    #mencari tag HTML 'b' yang merupakan tag dari title
    error = []
    title_tag = row.find('b')
    try:
        #jika title ada, maka
        if title_tag:
            #ambil data title
            title = title_tag.get_text()
            #ambil data link
            link = row['onclick'].split('=')[1].strip()
            #mengupdate bar agar bertambah
            bar.update(1)
    except:
        error.append(page)

    return {'title' : title, 'link' : "https://www.fortiguard.com" + link[1:-1]}

In [58]:
#Fungsi untuk membuat file dataset csv berdasarkan level
def into_csv(values, level):
    name = f"forti_lists_{level}.csv"
    error_csv = []

    try:
        #jika file csv telah terdapat pada folder, maka file tersebut yang akan digunakan
        df = pl.read_csv(name)
    except:
        data = {
            'title': [],
            'link': []
        }
        #jika file csv pada level tersebut belum ada, maka akan menggunakan file baru
        df = pl.DataFrame(data)

    #Menambahkan data title dan link pada dataframe untuk dimasukkan dalam file csv
    add_df = pl.DataFrame(values)

    #Memasukkan menambah data title dan link yang baru ke csv
    df.extend(add_df)
    df.write_csv(name)
    

In [59]:
#Fungsi untuk melakukan scraping data menggunakan library BeautifulSoup
def get_data(level, page, bar):
    url = f"https://www.fortiguard.com/encyclopedia?type=ips&risk={level}&page={page}"
    
    #Melakukan request untuk url website
    page = requests.get(url)

    #Melakukan parsing code HTML menjadi bentuk teks
    soup = BeautifulSoup(page.text, "html.parser")

    #Memilih tag HTML yang mengandung data penting yang dibutuhkan
    body = soup.select("section.table-body > div.container > div.row")

    #Mengambil data title dan link
    values = [get(row, bar, page, level) for row in body]

    #Memasukkan data pada file csv
    into_csv(values, level)

    

In [61]:
#Max pages sudah ditentukan berdasarkan jumlah page maksimal pada website
max_pages = [7, 34, 170, 386, 248]
#Total data tiap level sudah ditentukan dengan melihat data pada website
total_data = [127, 662, 3397, 7704, 4958]

data = []
#Perulangan untuk melakukan scraping data setiap level
for level in range(1, 6):
    #Barchart
    bar = tqdm(total_data[level-1])

    #Mengambil data title dan link tiap level sebanyak jumlah maksimal halamannya
    for page in range(1, max_pages[level-1]):
        get_data(level, page, bar)















20it [05:13, 15.68s/it]
20it [06:51, 20.56s/it]
20it [04:38, 13.92s/it]


